# Initialize

In [8]:
from ais import functions as af
from pyspark.sql import functions as F
from pyspark.sql.types import StringType

In [9]:
import h3.api.numpy_int as h3int
from shapely.geometry import mapping, Polygon, Point

from multiprocessing import Pool
import tqdm

In [10]:
import geopandas as gpd
import pandas as pd
import numpy as np

import folium

In [11]:
pd.set_option('display.max_columns', None) #Show all columns in pandas df
pd.set_option('display.max_rows', 100) #Show 100 rows in pandas df
pd.options.display.float_format = '{:.10f}'.format #Show float with 10 decimal points in pandas df

from IPython.core.interactiveshell import InteractiveShell #allow multiple outputs in one jupyter cell
InteractiveShell.ast_node_interactivity = "all"

In [12]:
#bucket = "ungp-ais-data-historical-backup"
#path = f"s3a://{bucket}/user_temp/adb/"
path = "s3a://ungp-ais-data-historical-backup/exact-earth-data/transformed/prod/"

# Buffer Polygons

In [13]:
def get_wpi():
    # wpi = gpd.read_file("https://msi.nga.mil/api/publications/download?key=16694622/SFH00000/WPI_Shapefile.zip")
    
    wpi = pd.read_csv("https://msi.nga.mil/api/publications/download?type=view&key=16920959/SFH00000/UpdatedPub150.csv") \
            [['World Port Index Number','Main Port Name','UN/LOCODE','Country Code','Harbor Size','Harbor Type','Latitude','Longitude']] \
            .rename(columns={'Country Code':'Country','Main Port Name':'Port'})
    
    geometry=gpd.points_from_xy(wpi['Longitude'],wpi['Latitude'])
    
    wpi = gpd.GeoDataFrame(wpi, geometry=geometry, crs="epsg:4326")
    return wpi

In [14]:
def poly_to_h3(dfseries, h3_res=8):
    return dfseries.apply(lambda x: h3int.polyfill(mapping(x), h3_res, geo_json_conformant=True))

def h3_to_poly(df_series, crs='epsg:4326'):
    return gpd.GeoSeries(df_series.apply(lambda x: Polygon(h3int.h3_set_to_multi_polygon(x, geo_json=True)[0][0])), crs=crs)

def parallelize_dataframe(df, func,n_split=100, n_cores=4):
    df_split = np.array_split(df, n_split)
    pool = Pool(n_cores) 
    mapped_values = list(tqdm.tqdm(pool.imap_unordered(func, df_split), total=n_split))
    pool.close()
    pool.join()
    return pd.concat(mapped_values).sort_index()

In [15]:
def wpi_buffer_combined(wpi, utm, buffer_dist_km=22, return_grouped=True):
    
    buffer_label = f"buffer_{buffer_dist_km}KM"

    print(f"WPI Rows: {wpi.shape[0]:,}")
    
    #aggregate UTM polygons per EPSG
    utm_agg = utm[~utm['ZONE'].isin(['A','B','Y','Z'])][['EPSG','geometry']].dissolve(by=['EPSG'],as_index=False)
    
    #attach UTM info to wpi if wpi polygon is in utm aggregated polygon
    df = gpd.sjoin(wpi[['Country','Port','World Port Index Number','geometry']].rename(columns={'World Port Index Number':'port_id'}),
                   utm_agg, predicate='within').drop(columns=['index_right'])
    print(f"WPI with points within UTM Zone: {df.shape[0]:,}")
    
    #generate buffer for each epsg batch
    base_crs = df.crs
    for epsg in df['EPSG'].unique():
        cond = df['EPSG'] == epsg
        df_small = df[cond].copy()
        df_small.to_crs(epsg, inplace=True)
        df_small[buffer_label] = df_small.buffer(buffer_dist_km*1000, cap_style=3)
        df_small.set_geometry(buffer_label, inplace=True)
        df_small["buffer_area"] = df_small.area
        df_small.to_crs(base_crs, inplace=True)

        df.loc[cond,[buffer_label, "buffer_area"]] = df_small[[buffer_label, "buffer_area"]]

    

#     #attach UTM info of buffer, this will cut the buffer into each utm
    df = df.set_geometry(buffer_label).rename(columns={"geometry":"location"})

    #combine overlapping buffer to create grouped buffer
    df2 = df[[buffer_label]].dissolve().explode(index_parts=True).reset_index(drop=True)
    df2['buffer_grouped_id'] = np.arange(df2.shape[0])
    df2[buffer_label+"_grouped"] = df2[buffer_label]
    
    print("Generating h3 indices for grouped buffer..")
    
    df2['buffer_grouped_h3'] = parallelize_dataframe(df2[buffer_label], poly_to_h3,100,4)
    
    print("Converting h3 indices to polygon..")
    df2['buffer_h3_poly'] = parallelize_dataframe(df2['buffer_grouped_h3'], h3_to_poly,100,4)
    
    df4 = df2[['buffer_grouped_h3','buffer_grouped_id']].explode('buffer_grouped_h3', ignore_index=True)
     
    #attach grouped buffer info
    df3 = gpd.sjoin(df.set_geometry("location"), 
          df2.drop(columns=['buffer_grouped_h3']),
          how='left',
          predicate='within').drop(columns=['index_right'])
    df3['grouped_country'] = df3['buffer_grouped_id'].map(df3.groupby('buffer_grouped_id')['Country'].agg(set))
    df3['grouped_port'] = df3['buffer_grouped_id'].map(df3.groupby('buffer_grouped_id')["Port"].agg(set))
    
    df3.set_geometry(buffer_label+"_grouped", inplace=True)
    df3.set_crs(epsg="4326", inplace=True)
    print("Done")
    return df3, df4

In [16]:
def get_utm():
    url = 'https://opendata.arcgis.com/datasets/b294795270aa4fb3bd25286bf09edc51_0.zip'
    utm = gpd.read_file(url)
    utm['UTM'] = utm['ZONE'].astype(str) + utm['ROW_']
    south = ['A','B','C','D','E','F','G','H','J','K','L','M']
    north = ['N','P','Q','R','S','T','U','V','W','X','Y','Z']
    utm['NS'] = np.where(utm['ROW_'].isin(north),"N","S")
    utm['prefix'] = np.where(utm['ROW_'].isin(north),"326","327")
    utm['EPSG'] = 'epsg:' + utm['prefix'] + utm['ZONE'].astype(str).str.zfill(2)
    return utm

## Call Func

In [17]:
wpi = get_wpi()
utm = get_utm()

In [18]:
wpi.info()
utm.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3831 entries, 0 to 3830
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   World Port Index Number  3831 non-null   float64 
 1   Port                     3831 non-null   object  
 2   UN/LOCODE                3831 non-null   object  
 3   Country                  3831 non-null   object  
 4   Harbor Size              3831 non-null   object  
 5   Harbor Type              3831 non-null   object  
 6   Latitude                 3831 non-null   float64 
 7   Longitude                3831 non-null   float64 
 8   geometry                 3831 non-null   geometry
dtypes: float64(3), geometry(1), object(5)
memory usage: 269.5+ KB
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1201 entries, 0 to 1200
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   FID       

In [19]:
#There are duplicated WPIs, keep first
wpi[wpi.duplicated(subset=['World Port Index Number'], keep=False)]

,World Port Index Number,Port,UN/LOCODE,Country,Harbor Size,Harbor Type,Latitude,Longitude,geometry
71,49460.0000000000,Machilipatnam,,India,Very Small,Open Roadstead,16.1500000000,81.1666670000,POINT (81.16667 16.15000)
1647,400.0000000000,Seydhisfjordhur,IS SEY,Iceland,Small,Coastal (Natural),65.2666670000,-14.0000000000,POINT (-14.00000 65.26667)
2552,49460.0000000000,Machilipatnam,,India,Very Small,Open Roadstead,16.1500000000,81.1500000000,POINT (81.15000 16.15000)
3769,400.0000000000,Seydisfjordur,ISSEY,Iceland,,,65.2666666670,-14.0000000000,POINT (-14.00000 65.26667)


In [20]:
wpi.drop_duplicates(subset=['World Port Index Number'], keep='first', inplace=True)

## Special Cases: 

In [21]:
#Drop ports
drop_ports = [43384, 44605, 59950, 59960, 29260, 15270, 28720, 28715, 8850] + [50010.0, 50000.0, 50017.0, 50015.0, 49995.0] + [59970]
drop_index = wpi[wpi['World Port Index Number'].isin(drop_ports)].index
wpi.loc[drop_index]
wpi.drop(index=drop_index, inplace=True)

,World Port Index Number,Port,UN/LOCODE,Country,Harbor Size,Harbor Type,Latitude,Longitude,geometry
88,43384.0000000000,Borusan Fertilizer Jetty,,Turkey,Very Small,Coastal (Natural),40.4166670000,29.1000000000,POINT (29.10000 40.41667)
124,28715.0000000000,Port Polnochny,,Poland,Medium,Coastal (Breakwater),54.4000000000,18.7166670000,POINT (18.71667 54.40000)
344,50010.0000000000,Pulau Bukom,SG PUB,Singapore,Medium,Open Roadstead,1.2333330000,103.7666670000,POINT (103.76667 1.23333)
534,29260.0000000000,Tuborg,DK TUB,Denmark,Very Small,Coastal (Breakwater),55.7166670000,12.5833330000,POINT (12.58333 55.71667)
754,28720.0000000000,Nowy Port,PL NWA,Poland,Small,Canal or Lake,54.4166670000,18.6666670000,POINT (18.66667 54.41667)
881,50000.0000000000,Keppel - (East Singapore),SG KEP,Singapore,Large,Coastal (Natural),1.2833330000,103.8500000000,POINT (103.85000 1.28333)
914,50017.0000000000,Jurong Island,SG JUR,Singapore,Large,Coastal (Natural),1.2833330000,103.7333330000,POINT (103.73333 1.28333)
1418,44605.0000000000,Kaba Burnu,,Turkey,Very Small,Coastal (Natural),40.7666670000,29.5333330000,POINT (29.53333 40.76667)
1885,8850.0000000000,Gretna,US GTL,United States,Very Small,River (Natural),29.9166670000,-90.0666670000,POINT (-90.06667 29.91667)
2291,59970.0000000000,Shanghai,CN SGH,China,Large,River (Natural),31.2166670000,121.5000000000,POINT (121.50000 31.21667)


In [22]:
# Change points
wpi.loc[wpi['Port']=="Itajai", ['Latitude','Longitude','geometry']] = [-26.90268, -48.6622, Point(-48.6622, -26.90268)]
wpi.loc[wpi['Port']=="Nemrut Limani Bay", ['Latitude','Longitude','geometry']] = [38.764729, 26.924815, Point(26.924815, 38.764729)]
wpi.loc[wpi['Port']=="Fos", ['Latitude','Longitude','geometry']] = [ 43.4229, 4.86513, Point( 4.86513, 43.4229)]
wpi.loc[wpi['Port']=="Gebze", ['Latitude','Longitude','geometry']] = [40.766667,	29.533333, Point (29.53333, 40.76667)]
wpi.loc[wpi['Port']=="Ningbo", ['Port','Latitude','Longitude','geometry']] = ["Ningbo-Zhoushan",29.948443, 122.049256, Point (122.049256,29.948443)]

In [23]:
# Multipoints

#Rotterdam
cond = wpi['World Port Index Number'].isin([31085, 31080,  31120, 31130, 31140, 31150])
wpi.loc[cond,"World Port Index Number"] = 31140
wpi.loc[cond,"Port"] = "Rotterdam"

#Ningbo Shoushan
wpi.loc[wpi['Port']=="Ningbo", ['Port','Latitude','Longitude','geometry']] = ["Ningbo-Zhoushan",29.948443, 122.049256, Point (122.049256,29.948443)]

In [24]:
#Add ports
n = wpi['World Port Index Number'].max()
add_ports = [
    [n+10000,"Yangshan Port","CN SGH","China","","",30.620378, 122.071496, Point( 122.071496,30.620378)],
    [59970 ,"Shanghai","CN SHG","China","","",31.409925, 121.508700, Point(121.508700,31.409925)],
    [59970 ,"Shanghai","CN SHG","China","","",31.322271, 121.659643, Point(121.659643,31.322271)],
    [n+10001, "Busan Port", "KR BNP","South Korea","","",35.0655, 128.799, Point( 128.799, 35.0655)],
    [n+10002,"Petkim Port", "TR ALI","Turkey","","",38.778640, 26.927955, Point(26.927955,38.778640)],
    [n+10003, "Solventas", "TR ", "Turkey","","",40.766761, 29.544444, Point(29.544444,40.766761)],
    [n+10004,"Posorja","EC PSJ" ,"Ecuador","","",  -2.708151, -80.24097, Point(-80.24097,  -2.708151)],
    [n+10005,"Navegantes", "BR NVT","Brazil","","", -26.89702, -48.66268, Point(-48.66268, -26.89702)],
    [n+10006,"Itapoa", "BR IOA","Brazil","","",-26.184, -48.6026, Point(-48.6026,-26.184)],
    [n+10007,"Hamad", "QA HMD","Qatar","","",25.02946, 51.6245, Point(51.6245,25.02946)],
    [n+10008,"Nansha", "CN NSA", "China", "","", 22.66917,113.6605, Point(113.6605, 22.66917)],
    [50000,"Singapore", "SG SIN","Singapore","","",1.264448, 103.682829, Point(103.682829,1.264448)],
    [50000,"Singapore", "SG SIN","Singapore","","",1.336188, 104.026898, Point(104.026898,1.336188)],
    [50000,"Singapore", "SG SIN","Singapore","","",1.291182, 103.916269, Point(103.916269,1.291182)],
    [50000,"Singapore", "SG SIN","Singapore","","",1.246654, 103.770679, Point(103.770679,1.246654)],
]

In [25]:
wpi = pd.concat([wpi,
           gpd.GeoDataFrame(add_ports, columns=wpi.columns, crs="epsg:4326")],ignore_index=True)

In [26]:
#Drop duplicates based on H3 resolution 8
wpi['h3'] = wpi[['Latitude','Longitude']].apply(lambda x: h3int.geo_to_h3(x[0],x[1],8), axis=1)
wpi[wpi.duplicated(subset=['h3'], keep=False)].sort_values(['h3'])

,World Port Index Number,Port,UN/LOCODE,Country,Harbor Size,Harbor Type,Latitude,Longitude,geometry,h3
2366,28490.0000000000,Paljassaare,EE PAS,Estonia,Medium,Coastal (Breakwater),59.4591020000,24.7056400000,POINT (24.70564 59.45910),612640945388650495
1691,28492.0000000000,Lahesuu,EE LHS,Estonia,Very Small,Coastal (Breakwater),59.4570562040,24.7030027230,POINT (24.70300 59.45706),612640945388650495
1189,19781.0000000000,Homer,US HOM,United States,Very Small,Coastal (Natural),59.6000000000,-151.4166670000,POINT (-151.41667 59.60000),612706276754849791
1527,19785.0000000000,Coal Point,,United States,Very Small,Coastal (Natural),59.6014000000,-151.4111000000,POINT (-151.41110 59.60140),612706276754849791
3321,19588.0000000000,Haines,US HNS,United States,Very Small,Coastal (Natural),59.2333330000,-135.4333330000,POINT (-135.43333 59.23333),612833771728666623
3556,19585.0000000000,Port Chilkoot,,United States,Very Small,Coastal (Natural),59.2333330000,-135.4333330000,POINT (-135.43333 59.23333),612833771728666623
181,31375.0000000000,Isle Of Grain,GB IOG,United Kingdom,Small,Coastal (Natural),51.4333330000,0.7000000000,POINT (0.70000 51.43333),612934725060788223
2135,31376.0000000000,Thamesport,GB THP,United Kingdom,Small,River (Natural),51.4333330000,0.7000000000,POINT (0.70000 51.43333),612934725060788223
371,16805.0000000000,Glenada,,United States,Very Small,River (Natural),43.9666670000,-124.1166670000,POINT (-124.11667 43.96667),613195105995587583
373,16803.0000000000,Florence,US FNE,United States,Very Small,River (Natural),43.9666670000,-124.1166670000,POINT (-124.11667 43.96667),613195105995587583


In [27]:
wpi.drop_duplicates(subset=['h3'], ignore_index=True, inplace=True)

In [28]:
wpi.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3814 entries, 0 to 3813
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   World Port Index Number  3814 non-null   float64 
 1   Port                     3814 non-null   object  
 2   UN/LOCODE                3814 non-null   object  
 3   Country                  3814 non-null   object  
 4   Harbor Size              3814 non-null   object  
 5   Harbor Type              3814 non-null   object  
 6   Latitude                 3814 non-null   float64 
 7   Longitude                3814 non-null   float64 
 8   geometry                 3814 non-null   geometry
 9   h3                       3814 non-null   int64   
dtypes: float64(3), geometry(1), int64(1), object(5)
memory usage: 298.1+ KB


## Save

In [31]:
save_path = "s3a://ungp-ais-data-historical-backup/user_temp/"
save_path_unique = save_path + "test/"

In [32]:
wpi.to_pickle(save_path_unique+"WPI.pkl")
utm.to_pickle(save_path_unique+"UTM.pkl")

## Generate buffers

In [33]:
#note, we parallellize conversion of polygons to h3 indices because it takes a while to convert in series, at least 5 mins for 1000K rows
ports_df, ports_grouped_df = wpi_buffer_combined(wpi, utm, buffer_dist_km=22)

WPI Rows: 3,814
WPI with points within UTM Zone: 3,814
Generating h3 indices for grouped buffer..
Closing down clientserver connection
Closing down clientserver connection
Closing down clientserver connection
Closing down clientserver connection


100%|██████████| 100/100 [00:04<00:00, 21.41it/s]

Closing down clientserver connection
Closing down clientserver connection
Converting h3 indices to polygon..


Closing down clientserver connection
Closing down clientserver connection
Closing down clientserver connection
Closing down clientserver connection


100%|██████████| 100/100 [00:12<00:00,  7.76it/s]

Closing down clientserver connection
Closing down clientserver connection


Done


/tmp/ipykernel_51/4079727764.py:48: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  df3 = gpd.sjoin(df.set_geometry("location"),


In [34]:
ports_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 3814 entries, 0 to 3696
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   Country              3814 non-null   object  
 1   Port                 3814 non-null   object  
 2   port_id              3814 non-null   float64 
 3   location             3814 non-null   geometry
 4   EPSG                 3814 non-null   object  
 5   buffer_22KM          3814 non-null   geometry
 6   buffer_area          3814 non-null   float64 
 7   buffer_grouped_id    3814 non-null   int64   
 8   buffer_22KM_grouped  3814 non-null   geometry
 9   buffer_h3_poly       3814 non-null   geometry
 10  grouped_country      3814 non-null   object  
 11  grouped_port         3814 non-null   object  
dtypes: float64(2), geometry(4), int64(1), object(5)
memory usage: 387.4+ KB


In [35]:
ports_df[ports_df['Country']=="Indonesia"]
#ports_df[ports_df['Country']=="Singapore"]['grouped_port'].iloc[0]

,Country,Port,port_id,location,EPSG,buffer_22KM,buffer_area,buffer_grouped_id,buffer_22KM_grouped,buffer_h3_poly,grouped_country,grouped_port
1,Indonesia,Mangkasa Oil Terminal,52235.0000000000,POINT (121.06667 -2.73333),epsg:32751,"POLYGON ((121.26477 -2.53469, 121.26420 -2.932...",1936000000.0000000000,1196,"POLYGON ((121.26477 -2.53469, 121.26420 -2.932...","POLYGON ((121.26260 -2.66655, 121.26330 -2.661...",{Indonesia},{Mangkasa Oil Terminal}
245,Indonesia,Kendari,52240.0000000000,POINT (122.58333 -3.96667),epsg:32751,"POLYGON ((122.78156 -3.76771, 122.78146 -4.165...",1936000000.0000000000,1193,"POLYGON ((122.78156 -3.76771, 122.78146 -4.165...","POLYGON ((122.64982 -4.16353, 122.65433 -4.161...",{Indonesia},{Kendari}
353,Indonesia,Pomalaa,52295.0000000000,POINT (121.60000 -4.16667),epsg:32751,"POLYGON ((121.79847 -3.96801, 121.79787 -4.365...",1936000000.0000000000,1191,"POLYGON ((121.79847 -3.96801, 121.79787 -4.365...","POLYGON ((121.67097 -4.36783, 121.67544 -4.366...",{Indonesia},{Pomalaa}
456,Indonesia,Kupang,51400.0000000000,POINT (123.58333 -10.16667),epsg:32751,"POLYGON ((123.78368 -9.96729, 123.78465 -10.36...",1936000000.0000000000,1178,"POLYGON ((123.78368 -9.96729, 123.78465 -10.36...","POLYGON ((123.76027 -10.36381, 123.76480 -10.3...",{Indonesia},{Kupang}
1306,Indonesia,Maumere,51320.0000000000,POINT (122.21667 -8.61667),epsg:32751,"POLYGON ((122.41690 -8.41804, 122.41629 -8.816...",1936000000.0000000000,1183,"POLYGON ((122.41690 -8.41804, 122.41629 -8.816...","POLYGON ((122.35769 -8.81768, 122.36216 -8.816...",{Indonesia},{Maumere}
...,...,...,...,...,...,...,...,...,...,...,...,...
3782,Indonesia,Kasim Terminal,53045.0000000000,POINT (131.01667 -1.30000),epsg:32752,"POLYGON ((131.21413 -1.10094, 131.21448 -1.498...",1936000000.0000000000,1315,"POLYGON ((131.21448 -1.49872, 131.18111 -1.498...","POLYGON ((131.04107 -1.10515, 131.03708 -1.101...",{Indonesia},"{Sailolof, Kasim Terminal, Salawati}"
3497,Indonesia,Jayapura,53133.0000000000,POINT (140.71667 -2.53333),epsg:32754,"POLYGON ((140.91458 -2.33432, 140.91455 -2.732...",1936000000.0000000000,1450,"POLYGON ((140.91458 -2.33432, 140.91455 -2.732...","POLYGON ((140.51561 -2.46092, 140.51961 -2.464...",{Indonesia},{Jayapura}
3770,Indonesia,Merauke,53130.0000000000,POINT (140.38333 -8.48333),epsg:32754,"POLYGON ((140.58342 -8.28460, 140.58299 -8.682...",1936000000.0000000000,1437,"POLYGON ((140.58342 -8.28460, 140.58299 -8.682...","POLYGON ((140.27577 -8.68071, 140.28064 -8.678...",{Indonesia},{Merauke}
2298,Indonesia,Uleelheue,50600.0000000000,POINT (95.28333 5.56667),epsg:32646,"POLYGON ((95.48265 5.76470, 95.48098 5.36703, ...",1936000000.0000000000,1102,"POLYGON ((95.11763 5.76616, 95.11883 6.08300, ...","POLYGON ((95.08454 5.71450, 95.08891 5.71156, ...",{Indonesia},"{Sabang, Uleelheue}"


In [36]:
ports_df = ports_df[ports_df['Country']=="Indonesia"]

In [48]:
# HHH started
#pip install geopandas

In [49]:
pip install geodatasets

Closing down clientserver connection
Note: you may need to restart the kernel to use updated packages.


In [44]:
import geopandas as gpd
import geodatasets
import folium
import matplotlib.pyplot as plt

generated new fontManager


In [45]:
m = folium.Map(location=[2, 120], zoom_start=4.2, tiles="CartoDB positron")
# latitude = -6.2088
# longitude = 106.8456
# folium.Marker([latitude, longitude], popup='Ini adalah titik').add_to(m)
m

In [46]:
for _, r in ports_df.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r["buffer_22KM"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "orange"})
    folium.Popup(r["Port"]).add_to(geo_j)
    geo_j.add_to(m)
    
m

In [47]:
for _, r in ports_df.iterrows():
    lat = r["location"].y
    lon = r["location"].x
    folium.Marker(
        location=[lat, lon],
        popup="length: {} <br> area: {}".format(r["buffer_area"], r["buffer_area"]),
    ).add_to(m)

m

In [27]:
# ports_df[ports_df['buffer_grouped_id'].isin(ports_df[ports_df['Port']=="Shanghai"].buffer_grouped_id)]

In [30]:
ports_df.to_pickle(path+"ki/wpi_22KM_v2.pkl")

In [31]:
for i in range(8,13):
    ports_df[f'H3_int_index_{i}'] = ports_df['location'].apply(lambda x: h3int.geo_to_h3(x.y, x.x, i))

In [32]:
ports_df[['port_id','H3_int_index_8','H3_int_index_9','H3_int_index_10','H3_int_index_11','H3_int_index_12']].drop_duplicates() \
    .to_parquet(path+"ki/global_point")

In [33]:
multiple_ports = ports_df[ports_df['grouped_port'].str.len() > 1]['buffer_grouped_id'].unique()

In [34]:
ports_grouped_df = ports_grouped_df.merge(
    ports_df[~ports_df['buffer_grouped_id'].isin(multiple_ports)][['buffer_grouped_id','port_id']],
    on=['buffer_grouped_id'],
    how='left')

In [35]:
ports_grouped_df.info()
ports_grouped_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7304278 entries, 0 to 7304277
Data columns (total 3 columns):
 #   Column             Dtype  
---  ------             -----  
 0   buffer_grouped_h3  object 
 1   buffer_grouped_id  int64  
 2   port_id            float64
dtypes: float64(1), int64(1), object(1)
memory usage: 222.9+ MB


,buffer_grouped_h3,buffer_grouped_id,port_id
0,615323611256848383,0,NaN
1,615323620899553279,0,NaN
2,615323654649020415,0,NaN
3,615323622252216319,0,NaN
4,615323613060399103,0,NaN


In [36]:
ports_grouped_df.to_parquet(f"{path}ki/wpi_22KM_grouped/")

# Manually drawn boundaries for passageways

drawn from https://geojson.io/

## babel 

In [105]:
babel1 = Polygon([
          [
            43.27895975093108,
            12.479248963885425
          ],
          [
            43.60145612579805,
            12.755308659453817
          ],
          [
            44.08237177252951,
            12.589708545824834
          ],
          [
            43.38645854255341,
            11.98160302124407
          ],
          [
            43.27895975093108,
            12.479248963885425
          ]
        ]
)

In [106]:
babel2 = Polygon([
          [
            42.368054706734966,
            13.38358837752314
          ],
          [
            43.15449323491927,
            13.944351521664089
          ],
          [
            42.9621269762267,
            14.45445207099813
          ],
          [
            42.03990050073,
            13.763074483095224
          ],
          [
            42.368054706734966,
            13.38358837752314
          ],
        ])

## Hormuz 

In [107]:
hormuz1 =Polygon( [
          [
            56.54209088104923,
            26.345701990099286
          ],
          [
            57.052493823986566,
            26.44681101991337
          ],
          [
            57.18991000093121,
            26.094746776191414
          ],
          [
            56.48074437348609,
            25.98230244908771
          ],
          [
            56.54209088104923,
            26.345701990099286
          ]
        ])

In [108]:
hormuz2 = Polygon([[
            56.14664785983311,
            26.144080251788964
          ],
          [
            55.53733355176945,
            26.77871964243961
          ],
          [
            55.15065331780599,
            26.723786588768462
          ],
          [
            55.97674290854613,
            25.83600264889769
          ],
          [
            56.14664785983311,
            26.144080251788964
          ]])

## Bering

In [109]:
bering1 = Polygon([
          [
            -170.73805004714242,
            65.56530230079281
          ],
          [
            -167.4732181822378,
            65.33405060664742
          ],
          [
            -166.26813960465665,
            64.455343087779
          ],
          [
            -172.2669123721415,
            64.67003594428746
          ],
          [
            -170.73805004714242,
            65.56530230079281
          ]])

In [110]:
bering2 = Polygon([
          [
            -170.35531957639424,
            66.36461724180995
          ],
          [
            -167.20860235541656,
            65.92759008186346
          ],
          [
            -165.30051047721545,
            66.44104425186944
          ],
          [
            -171.618533288113,
            66.97481018244048
          ],
          [
            -170.35531957639424,
            66.36461724180995
          ]
        ])

## png

In [111]:
png1 = Polygon([
          [
            143.54747970550113,
            -8.959293027678143
          ],
          [
            142.89046905873647,
            -11.405714245437778
          ],
          [
            143.62825963140688,
            -12.783360006146125
          ],
          [
            145.26514781705595,
            -7.845220282519449
          ],
          [
            143.52183304603545,
            -8.934225968514397
          ],
          [
            143.48343990990293,
            -8.934472529535554
          ]
        ])

In [112]:
png2 = Polygon([
          [
            141.39271814215806,
            -9.189806079027534
          ],
          [
            142.09358699203204,
            -11.11538407055042
          ],
          [
            141.6541520914178,
            -12.345911576286795
          ],
          [
            139.92154089960428,
            -8.231751508509248
          ],
          [
            141.379938290852,
            -9.21494394104458
          ]
        ])

## japan

In [113]:
jpn1 = Polygon([
          [
            142.2350265373035,
            45.273705203409946
          ],
          [
            142.20296982512048,
            46.0911353961655
          ],
          [
            143.4291390661225,
            46.1910875705841
          ],
          [
            143.29289803934455,
            44.39853853651127
          ],
          [
            142.2350265373035,
            45.26242459433013
          ]
        ])

In [114]:
jpn2 = Polygon([
          [
            141.8856631131332,
            46.13701668299231
          ],
          [
            141.6131810595772,
            45.30899071377837
          ],
          [
            140.8117632550007,
            45.263882887638744
          ],
          [
            140.77970654281762,
            46.66755482490876
          ],
          [
            141.90169146922472,
            46.13701668299231
          ]
        ])

## Japan South Korea

In [115]:
sokor1 = Polygon([
          [
            129.33668711989776,
            35.421842495600686
          ],
          [
            130.9810206803253,
            34.362152214650436
          ],
          [
            132.39044944640608,
            35.140196165223344
          ],
          [
            129.46882106671785,
            36.762381401369495
          ],
          [
            129.38073176883773,
            35.4038943526013
          ]
        ])

In [116]:
sokor2 = Polygon([
          [
            127.91991870561918,
            34.712878558134406
          ],
          [
            129.44679963162667,
            33.27661791984407
          ],
          [
            128.55122510317955,
            32.5865643647203
          ],
          [
            126.88486921828195,
            34.174092141396684
          ],
          [
            127.91991870561918,
            34.712878558134406
          ]
        ])

In [117]:
sg1 = Polygon([
          [
            104.24203225859111,
            1.6505628383036992
          ],
          [
            104.2788192333881,
            1.3715119639307147
          ],
          [
            104.38918015777926,
            1.1832960825392007
          ],
          [
            104.62937746380851,
            1.1616612164212796
          ],
          [
            104.24203225859111,
            1.6505628383036992
          ]
        ])

In [118]:
sg2 = Polygon([
          [
            103.15560681340821,
            0.9016872915739782
          ],
          [
            103.49289539213794,
            1.284020633524463
          ],
          [
            103.34895695408125,
            1.5503324363120328
          ],
          [
            102.99448169618063,
            1.0842682575379996
          ],
          [
            103.15560681340821,
            0.9016872915739782
          ]
        ])

In [119]:
sg3 = Polygon(
    [
          [
            98.23683440342586,
            4.4179433814712326
          ],
          [
            100.39630762290955,
            5.682757950093304
          ],
          [
            99.722551978431,
            6.901925781850082
          ],
          [
            97.13982200792748,
            5.27003248678588
          ],
          [
            98.23683440342586,
            4.4179433814712326
          ]
        ]
)

In [120]:
# Danish
db1 = Polygon(
     [
          [
            10.447154042413331,
            56.548141239881005
          ],
          [
            12.428792950137222,
            56.89586359969775
          ],
          [
            11.872694559580765,
            57.745245752027785
          ],
          [
            10.435630899003229,
            57.52269766661777
          ],
          [
            10.447154042413331,
            56.548141239881005
          ]
        ])
db2 = Polygon(
    [
          [
            13.115907618875838,
            55.37982784568749
          ],
          [
            12.858896486789348,
            54.34444711605286
          ],
          [
            14.315292901946151,
            53.96820165710426
          ],
          [
            14.279596911378064,
            55.436571345079244
          ],
          [
             13.115907618875838,
            55.37982784568749
          ]
        ])

## All

In [121]:
passthru_manual = gpd.GeoDataFrame([
    [babel1,"Bab El-Mandeb SE", "Bab El-Mandeb Strait",1,1],
    [babel2,"Bab El-Mandeb NW", "Bab El-Mandeb Strait",2,1],
    [hormuz1,"Hormuz E","Strait of Hormuz",1,2],
    [hormuz2,"Hormuz W","Strait of Hormuz",2,2],
    [bering1, "Bering S","Bering Strait",1,3],
    [bering2,"Bering N","Bering Strait", 2,3],
    [png1, "Torres E", "Torres Strait",1,4],
    [png2, "Torres W", "Torres Strait",2,4],
    [jpn1, "La Pérouse E","La Pérouse Strait",1,5],
    [jpn2, "La Pérouse W","La Pérouse Strait",2,5],
    [sokor1, "Korea NE", "Korea Strait", 1, 6],
    [sokor2,"Korea SW", "Korea Strait", 2, 6],
    [sg1,"Singapore 1", "Singapore Strait", 1, 17],
    [sg2,"Singapore 2", "Singapore Strait", 2, 17],
    [sg3,"Singapore 3", "Singapore Strait", 3, 17],
    [db1,"Danish N","Danish Straits",1,18],
    [db2,"Danish S","Danish Straits",2,18]
    
],
columns=['geometry','Passage_Part','Passage','passage_part_id','passage_id'],
crs="epsg:4326")

passthru_manual.to_pickle(f"{path}ki/Passthru.pkl")

In [122]:
passthru_manual['h3'] = poly_to_h3(passthru_manual.geometry)
passthru_h3 = passthru_manual[['h3','passage_part_id','passage_id']].explode("h3", ignore_index=True)

In [123]:
passthru_h3.to_parquet(f"{path}ki/passage_manual/")

In [124]:
passthru_h3 = None

In [125]:
# ports_df[ports_df['buffer_grouped_id'].isin(passthru_buffer['buffer_grouped_id'])][['Country','Port','buffer_grouped_id']].drop_duplicates()

In [126]:
passthru_buffer = pd.DataFrame([
    [868, "Suez Canal Anchorage N", "Suez Canal",1,7],
    [864, "Suez Canal Anchorage S", "Suez Canal",2,7],
    [865, "Suez Canal Great Bitter Lake", "Suez Canal",3,7],
    [579, "Gibraltar Strait","Gibraltar Strait",1,8],
    [642,"English Channel","English Channel",1,10],
    [885,"Bosphorus Strait","Bosphorus Strait",1,11],
    [798, "Dardanelles Strait", "Dardanelles Strait", 1,12],
    [730, "Cape of Good Hope", "Cape of Good Hope", 1,13],
    [213, "Panama Canal N", "Panama Canal",1,14],
    [211,"Panama Canal S", "Panama Canal",2,14],
    [309, "Magellan Strait N", "Magellan Strait", 1,15],
    [306, "Magellan Strait S", "Magellan Strait", 2, 15],
], columns = ["buffer_grouped_id","Passage Part", "Passage","passage_part_id","passage_id"]
)
    


In [127]:
passthru_buffer.to_pickle(f"{path}ki/Passthru_Buffer.pkl")

# Overlapping ports reference

In [128]:
ports_df = pd.read_pickle(path+"ki/wpi_22KM_v2.pkl")
multiple_ports = ports_df[ports_df['grouped_port'].apply(lambda x: len(x)) > 1].buffer_grouped_id.unique()

In [129]:
ports_overlap = ports_df[ports_df['buffer_grouped_id'].isin(multiple_ports)].set_index(['buffer_grouped_id','port_id'])['buffer_22KM']
ports_overlap_h3 = parallelize_dataframe(ports_overlap, poly_to_h3,100,4) \
                        .reset_index() \
                        .explode("buffer_22KM", ignore_index=True)

Closing down clientserver connection
Closing down clientserver connection
Closing down clientserver connection
Closing down clientserver connection


100%|██████████| 100/100 [00:05<00:00, 19.95it/s]


In [130]:
ports_overlap_h3.to_parquet(f"{path}ki/overlapping/")

# Combine All

In [133]:
#all buffer grouped id with multiple ports

overlap_sdf = spark.read.parquet(f"{path}ki/overlapping/")
overlap_sdf.printSchema()
overlap_sdf.count()
overlap_sdf.show()

root
 |-- buffer_grouped_id: long (nullable = true)
 |-- port_id: double (nullable = true)
 |-- buffer_22KM: decimal(20,0) (nullable = true)



8159729

+-----------------+-------+------------------+
|buffer_grouped_id|port_id|       buffer_22KM|
+-----------------+-------+------------------+
|                7|55770.0|614902042860716031|
|                7|55770.0|614899994710769663|
|                7|55770.0|614902026330963967|
|                7|55770.0|614900011240521727|
|                7|55770.0|614902044999811071|
|                7|55770.0|614902044708306943|
|                7|55770.0|614899992571674623|
|                7|55770.0|614899988876492799|
|                7|55770.0|614899995002273791|
|                7|55770.0|614899988293484543|
|                7|55770.0|614902044416802815|
|                7|55770.0|614899991015587839|
|                7|55770.0|614900010657513471|
|                7|55770.0|614902042277707775|
|                7|55770.0|614899995392344063|
|                7|55770.0|614899993154682879|
|                7|55770.0|614899993253249023|
|                7|55770.0|614899991114153983|
|            

In [134]:
overlap_sdf.select(F.countDistinct("buffer_22KM")).show()

+---------------------------+
|count(DISTINCT buffer_22KM)|
+---------------------------+
|                    4808538|
+---------------------------+



In [135]:
overlap_agg_sdf = \
overlap_sdf.withColumnRenamed("buffer_22KM","H3_int_index_8") \
            .groupBy("H3_int_index_8") \
            .agg(F.collect_set("port_id").alias("port_id_list"),
                 F.first("buffer_grouped_id").alias("buffer_grouped_id"),
                 F.countDistinct("port_id").alias("port_count")
                ) \
            .withColumn("port_id", F.when(F.col("port_count")==1, F.col("port_id_list").getItem(0)))
overlap_agg_sdf.printSchema()
overlap_agg_sdf.count()

root
 |-- H3_int_index_8: decimal(20,0) (nullable = true)
 |-- port_id_list: array (nullable = false)
 |    |-- element: double (containsNull = false)
 |-- buffer_grouped_id: long (nullable = true)
 |-- port_count: long (nullable = false)
 |-- port_id: double (nullable = true)



4808538

In [136]:
overlap_agg_sdf.show(n=10)

+------------------+------------+-----------------+----------+-------+
|    H3_int_index_8|port_id_list|buffer_grouped_id|port_count|port_id|
+------------------+------------+-----------------+----------+-------+
|612509340533784575|   [20910.0]|              841|         1|20910.0|
|612509340542173183|   [20910.0]|              841|         1|20910.0|
|612509340546367487|   [20910.0]|              841|         1|20910.0|
|612509340561047551|   [20910.0]|              841|         1|20910.0|
|612509340571533311|   [20910.0]|              841|         1|20910.0|
|612509340577824767|   [20910.0]|              841|         1|20910.0|
|612509340579921919|   [20910.0]|              841|         1|20910.0|
|612509340600893439|   [20910.0]|              841|         1|20910.0|
|612509340634447871|   [20910.0]|              841|         1|20910.0|
|612509340642836479|   [20910.0]|              841|         1|20910.0|
+------------------+------------+-----------------+----------+-------+
only s

In [137]:
#manually drawn polygons for passageways
passthru_sdf = spark.read.parquet(f"{path}ki/passage_manual/") \
                    .withColumnRenamed("h3","H3_int_index_8") \
                    .withColumnRenamed("passage_part_id","passage_part_id_manual") \
                    .withColumnRenamed("passage_id","passage_id_manual")
passthru_sdf.printSchema()
passthru_sdf.count()

root
 |-- H3_int_index_8: decimal(20,0) (nullable = true)
 |-- passage_part_id_manual: long (nullable = true)
 |-- passage_id_manual: long (nullable = true)



421193

In [138]:
passthru_sdf.select(F.countDistinct("H3_int_index_8")).show()

+------------------------------+
|count(DISTINCT H3_int_index_8)|
+------------------------------+
|                        421193|
+------------------------------+



In [140]:
passthru_buffer_sdf = spark.createDataFrame(pd.read_pickle(f"{path}ki/Passthru_Buffer.pkl") \
                                               [['buffer_grouped_id','passage_part_id','passage_id']] \
                                                .rename(columns={'passage_part_id':'passage_part_id_buffer',
                                                                 'passage_id':'passage_id_buffer'}
                                                       )
                                           )
passthru_buffer_sdf.printSchema()
passthru_buffer_sdf.count()

root
 |-- buffer_grouped_id: long (nullable = true)
 |-- passage_part_id_buffer: long (nullable = true)
 |-- passage_id_buffer: long (nullable = true)



/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


12

In [141]:
#22KM buffer grouped, with port ids attached for buffers with single port
grouped_sdf = spark.read.parquet(f"{path}ki/wpi_22KM_grouped/") \
                    .select("buffer_grouped_h3","buffer_grouped_id","port_id") \
                    .withColumnRenamed("buffer_grouped_h3","H3_int_index_8")
grouped_sdf.printSchema()
grouped_sdf.count()

root
 |-- H3_int_index_8: decimal(20,0) (nullable = true)
 |-- buffer_grouped_id: long (nullable = true)
 |-- port_id: double (nullable = true)



7304278

In [142]:
grouped_sdf.select("H3_int_index_8","port_id").distinct().count()

7304278

In [143]:
grouped_sdf.select(F.countDistinct("H3_int_index_8")).show()

+------------------------------+
|count(DISTINCT H3_int_index_8)|
+------------------------------+
|                       7304278|
+------------------------------+



In [144]:
grouped_sdf.groupby("H3_int_index_8").count().filter(F.col("count")>1).show(n=10)

+--------------+-----+
|H3_int_index_8|count|
+--------------+-----+
+--------------+-----+



In [73]:
grouped_sdf.select(F.max("buffer_grouped_id")).show()

+----------------------+
|max(buffer_grouped_id)|
+----------------------+
|                  1508|
+----------------------+



In [74]:
combined_sdf =grouped_sdf \
            .join(overlap_agg_sdf.drop("buffer_grouped_id").withColumnRenamed("port_id","single_port_id"), #buffer with multiple port, fill port id for hexes with single port
                   on="H3_int_index_8",
                   how="left"
                  ) \
            .join(passthru_sdf, #manually drawn passageway
                   on = "H3_int_index_8",
                   how="outer"
                  ) \
            .join(passthru_buffer_sdf, #buffers with passageways
                  on = "buffer_grouped_id",
                  how = "left"
                 ) \
            .withColumn("port_id",F.coalesce("port_id","single_port_id")) \
            .withColumn("passage_part_id",F.coalesce("passage_part_id_manual","passage_part_id_buffer")) \
            .withColumn("passage_id", F.coalesce("passage_id_manual","passage_id_buffer")) \
            .withColumn("passage_id_temp", F.lit(2000) + F.col("passage_id")) \
            .withColumn("buffer_grouped_id", F.coalesce("buffer_grouped_id","passage_id_temp")) \
            .drop("single_port_id",
                  "passage_part_id_manual","passage_part_id_buffer","passage_id_manual","passage_id_buffer", "passage_id_temp") 

combined_sdf.printSchema()
combined_sdf.count()

root
 |-- buffer_grouped_id: long (nullable = true)
 |-- H3_int_index_8: decimal(20,0) (nullable = true)
 |-- port_id: double (nullable = true)
 |-- port_id_list: array (nullable = true)
 |    |-- element: double (containsNull = false)
 |-- port_count: long (nullable = true)
 |-- passage_part_id: long (nullable = true)
 |-- passage_id: long (nullable = true)



7694797

In [75]:
#should be same as count above, i.e. no duplicate hex
combined_sdf.select(F.countDistinct("H3_int_index_8")).show()

+------------------------------+
|count(DISTINCT H3_int_index_8)|
+------------------------------+
|                       7694797|
+------------------------------+



In [76]:
#all hexes have buffer_grouped_id, even the passageways
combined_sdf.filter(F.col("buffer_grouped_id").isNotNull()).count()

7694797

In [77]:
#suez canal
combined_sdf.filter(F.col("passage_id")==7).select("buffer_grouped_id","port_id","port_id_list","passage_id","passage_part_id").distinct().show()

+-----------------+-------+------------------+----------+---------------+
|buffer_grouped_id|port_id|      port_id_list|passage_id|passage_part_id|
+-----------------+-------+------------------+----------+---------------+
|              864|   null|[48120.0, 48121.0]|         7|              2|
|              865|   null|[47970.0, 47974.0]|         7|              3|
|              865|47974.0|         [47974.0]|         7|              3|
|              868|48104.0|         [48104.0]|         7|              1|
|              864|48121.0|         [48121.0]|         7|              2|
|              868|48106.0|         [48106.0]|         7|              1|
|              864|48120.0|         [48120.0]|         7|              2|
|              868|   null|[48104.0, 48106.0]|         7|              1|
|              868|   null|[48108.0, 48106.0]|         7|              1|
|              865|47970.0|         [47970.0]|         7|              3|
|              868|48108.0|         [4

In [78]:
combined_sdf.filter(F.col("passage_id")==6).select("buffer_grouped_id","port_id","port_id_list","passage_id","passage_part_id").distinct().show()

+-----------------+-------+------------------+----------+---------------+
|buffer_grouped_id|port_id|      port_id_list|passage_id|passage_part_id|
+-----------------+-------+------------------+----------+---------------+
|             2006|   null|              null|         6|              1|
|             2006|   null|              null|         6|              2|
|             1349|   null|[60370.0, 60376.0]|         6|              2|
|             1349|60370.0|         [60370.0]|         6|              2|
|             1352|60400.0|         [60400.0]|         6|              1|
|             1357|60410.0|              null|         6|              1|
|             1338|62340.0|              null|         6|              2|
|             1340|61720.0|         [61720.0]|         6|              1|
|             1350|61730.0|              null|         6|              1|
+-----------------+-------+------------------+----------+---------------+



In [79]:
combined_sdf.write.mode("overwrite").parquet(f"{path}ki/global_polygon/")